In [1]:
from itertools import combinations
from itertools import permutations

# C2
def make_candidate(freq_itemsets, k):
    candidates = set()
    for itemset1 in freq_itemsets:
        for itemset2 in freq_itemsets:
            union = itemset1 | itemset2
            if len(union) == k:
                for item in union: # 각각의 부분집합이 freq_itemsets에 포함되는지 확인
                    if union - {item} not in freq_itemsets:
                        break
                else:
                    candidates.add(union)

    return candidates

# 일반화
def filter(candidates, k, s):
    itemset_cnt_k = {}
    with open("../data/groceries.csv", "r") as f:
        for line in f:
            basket = line.strip().split(",")
            for comb in combinations(basket, k):
                comb = frozenset(comb)
                if comb in candidates:
                    if comb not in itemset_cnt_k:
                        itemset_cnt_k[comb] = 0
                    itemset_cnt_k[comb] += 1

    freq_itemsets = set(itemset for itemset, cnt in itemset_cnt_k.items() if cnt >= s)

    return freq_itemsets

In [2]:
# item 갯수 구하기
s = 100

item_cnt = {}

with open("../data/groceries.csv", "r") as f:
    for line in f:
        basket = line.strip().split(",")
        for item in basket:
            if item not in item_cnt:
                item_cnt[item] = 0
            item_cnt[item] += 1


# L1
freq_itemsets = set(frozenset([item]) for item, cnt in item_cnt.items() if cnt >= s)

freq_itemsets_all = freq_itemsets.copy()

In [3]:
k = 2

while len(freq_itemsets) > 0:
    candidates = make_candidate(freq_itemsets, k)
    freq_itemsets = filter(candidates, k, s)
    freq_itemsets_all |= freq_itemsets
    print(k, len(candidates), len(freq_itemsets))
    k += 1

for fi in freq_itemsets_all:
    print(fi)

2 3828 207
3 548 31
4 6 0
frozenset({'bottled water', 'bottled beer'})
frozenset({'soda', 'whipped/sour cream'})
frozenset({'frozen meals'})
frozenset({'tropical fruit', 'other vegetables'})
frozenset({'grapes'})
frozenset({'UHT-milk'})
frozenset({'meat'})
frozenset({'tropical fruit', 'other vegetables', 'root vegetables'})
frozenset({'coffee', 'whole milk'})
frozenset({'soda', 'napkins'})
frozenset({'tropical fruit', 'whole milk'})
frozenset({'pip fruit'})
frozenset({'other vegetables', 'shopping bags'})
frozenset({'root vegetables', 'citrus fruit'})
frozenset({'butter milk'})
frozenset({'citrus fruit', 'domestic eggs'})
frozenset({'frankfurter', 'other vegetables'})
frozenset({'onions', 'other vegetables'})
frozenset({'root vegetables', 'butter'})
frozenset({'canned fish'})
frozenset({'yogurt', 'brown bread'})
frozenset({'root vegetables', 'whole milk'})
frozenset({'domestic eggs', 'whole milk'})
frozenset({'whole milk', 'beef'})
frozenset({'pastry'})
frozenset({'cat food'})
frozense

In [4]:
lst = []

with open("../data/groceries.csv", "r") as f:
    for line in f:
        basket = line.strip().split(",")
        lst.append(basket)

In [5]:
dict_item_cnt = {}

for fi in freq_itemsets_all:
    for i in range(1, len(fi)+1):
        for comb in combinations(fi,i):
            comb = sorted(list(comb))
            if tuple(comb) in dict_item_cnt:
                continue
            for basket in lst:
                cnt = 0
                size = len(comb)
                for element in comb:
                    if element in basket:
                        cnt += 1
                if cnt == size:
                    comb = tuple(list(comb))
                    if comb not in dict_item_cnt:
                        dict_item_cnt[comb] = 1
                    else:
                        dict_item_cnt[comb] += 1
                        
# for key, value in dict_item_cnt.items():
#     print(key, value)

In [6]:
case = set()
c = 0.5

for fi in freq_itemsets_all:
    for i in range(1, len(fi)):
        for comb1 in combinations(fi,i):
            antecedent = tuple(sorted(list(comb1)))
            diff = fi.difference(antecedent)
            for j in range(1, len(diff)+1):
                for comb2 in combinations(diff,j):
                    consequent = tuple(sorted(list(comb2)))
                    union = tuple(sorted(list(set(list(antecedent)+list(consequent)))))
                    conf = dict_item_cnt[union]/dict_item_cnt[antecedent]
                    if conf >= c:
                        case.add((antecedent, '->', consequent, conf))
                        
for item in case:
    print(item)

(('root vegetables', 'yogurt'), '->', ('other vegetables',), 0.5)
(('other vegetables', 'pip fruit'), '->', ('whole milk',), 0.5175097276264592)
(('other vegetables', 'yogurt'), '->', ('whole milk',), 0.5128805620608899)
(('domestic eggs', 'other vegetables'), '->', ('whole milk',), 0.5525114155251142)
(('citrus fruit', 'root vegetables'), '->', ('other vegetables',), 0.5862068965517241)
(('whipped/sour cream', 'yogurt'), '->', ('whole milk',), 0.5245098039215687)
(('rolls/buns', 'root vegetables'), '->', ('whole milk',), 0.5230125523012552)
(('butter', 'other vegetables'), '->', ('whole milk',), 0.5736040609137056)
(('other vegetables', 'whipped/sour cream'), '->', ('whole milk',), 0.5070422535211268)
(('root vegetables', 'tropical fruit'), '->', ('other vegetables',), 0.5845410628019324)
(('tropical fruit', 'yogurt'), '->', ('whole milk',), 0.5173611111111112)
(('root vegetables', 'tropical fruit'), '->', ('whole milk',), 0.5700483091787439)
(('root vegetables', 'yogurt'), '->', ('wh

In [7]:
print(len(case))

14
